# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(pd.NaT, psycopg2._psycopg.AsIs)


In [2]:

conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("./data/song_data/")

In [6]:
df=pd.read_json(sorted(song_files)[0], lines=True)

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[['song_id', 'title', 'artist_id','year', 'duration']].values[0].tolist()

#check types for numpy types not supported in postgres
[type(x) for x in song_data]


[str, str, str, int, float]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id','artist_name', 'artist_location','artist_latitude', 'artist_longitude']].values.tolist()[0]
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("./data/log_data/")

In [ ]:
filepath = log_files[0]

In [12]:
df = pd.read_json(filepath, lines=True)
df


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
386,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
df = df.where(df['page'] == 'NextSong')
df = df.dropna(subset=['song', "ts", 'artist', 'userId'])
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0.0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829.0,Jim Henson's Dead,200.0,1.543537e+12,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0.0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049.0,Shell Shock,200.0,1.543540e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1.0,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049.0,Women Lose Weight (Feat: Slick Rick),200.0,1.543540e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2.0,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049.0,Won't Go Home Without You,200.0,1.543541e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3.0,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049.0,Hey_ Soul Sister,200.0,1.543541e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,Foo Fighters,Logged In,Rylan,M,57.0,George,271.38567,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076.0,The Pretender,200.0,1.543603e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
383,Timbiriche,Logged In,Rylan,M,58.0,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076.0,Besos De Ceniza,200.0,1.543603e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
384,A Perfect Circle,Logged In,Rylan,M,59.0,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076.0,Rose,200.0,1.543604e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
385,Anberlin,Logged In,Rylan,M,60.0,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076.0,The Haunting,200.0,1.543604e+12,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


In [14]:
#df = df.dropna()
#df

In [15]:
t = pd.to_datetime(df['ts'], unit='ms')
t.dt.isocalendar()

,year,week,day
0,2018,48,5
1,2018,48,5
2,2018,48,5
3,2018,48,5
4,2018,48,5
...,...,...,...
382,2018,48,5
383,2018,48,5
384,2018,48,5
385,2018,48,5


In [16]:
#timestamp, hour, day, week of year, month, year, and weekday
time_data = [t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.isocalendar().year, t.dt.isocalendar().day]
column_labels = ['timestamp', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']

In [17]:
time_df = pd.DataFrame(data=dict(zip(column_labels, time_data)))

time_df = time_df.dropna()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    try:
        cur.execute(time_table_insert, list(row))
        conn.commit()
    except:
        print(row)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit() 

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
#timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [22]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    #(time_stamp, user_id, level, song_id, artist_id, session_id, location, user_agent)
    
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

## Explore data from sparkify db


In [23]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()
df_spak = pd.read_sql_query("""SELECT * from songplays""", conn)


In [24]:
df_spak.sample(frac=0.1)

,songplay_id,time_stamp,user_id,level,song_id,artist_id,session_id,location,user_agent
211,212,2018-11-30 14:49:37.796,16,paid,None,None,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
48,49,2018-11-30 04:42:05.796,49,paid,None,None,1079,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
252,253,2018-11-30 16:00:31.796,16,paid,None,None,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
325,326,2018-11-30 18:40:05.796,16,paid,None,None,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
275,276,2018-11-30 16:49:06.796,16,paid,None,None,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
165,166,2018-11-30 13:27:49.796,49,paid,None,None,1096,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
319,320,2018-11-30 18:29:02.796,16,paid,None,None,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
179,180,2018-11-30 13:59:31.796,50,free,None,None,1050,"New Haven-Milford, CT","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
273,274,2018-11-30 16:43:28.796,33,free,None,None,1026,"Eugene, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
294,295,2018-11-30 17:39:17.796,16,paid,None,None,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
